# Check mining polygons

In [1]:
import geopandas as gpd
import shapely
import leafmap.foliumap as leafmap
import rasterio
from shapely.geometry import box

# optional change working directiory first
import os
print(os.getcwd())
os.chdir("mine-segmentation")

import sys
sys.path.append('..')

from src.data.get_satellite_images import ReadSTAC

/teamspace/studios/this_studio


In [24]:
# download mining polygons
# !python src/data/get_mining_polygons.py

In [2]:
MAUS_POLYGONS = "data/external/maus_mining_polygons.gpkg"
MAUS_AREA_RASTER = "data/external/global_miningarea_v2_5arcminute.tif"
TANG_POLYGONS = "data/external/tang_mining_polygons/74548_mine_polygons/74548_projected.shp"

# filter both dataframes to only the area of interest
LOCATION = [-50.16556135114535, -6.060451692157381]
BBOX = [LOCATION[0] - 0.5, LOCATION[1] - 0.5, LOCATION[0] + 0.5, LOCATION[1] + 0.5]


In [4]:
# Load a GeoPackage file into a GeoDataFrame
maus_gdf = gpd.read_file(MAUS_POLYGONS)

# Load a Shapefile into a GeoDataFrame
tang_gdf = gpd.read_file(TANG_POLYGONS)

In [5]:
maus_gdf.head()

,ISO3_CODE,COUNTRY_NAME,AREA,geometry
0,GBR,United Kingdom,1.782044,"POLYGON ((-1.67130 55.07660, -1.67040 55.07710..."
1,FJI,Fiji,0.023405,"POLYGON ((177.85480 -17.51730, 177.85490 -17.5..."
2,NZL,New Zealand,0.042758,"POLYGON ((175.87200 -37.34490, 175.87180 -37.3..."
3,FJI,Fiji,0.248552,"POLYGON ((177.84260 -17.51850, 177.84370 -17.5..."
4,FJI,Fiji,0.132724,"POLYGON ((177.84950 -17.49510, 177.84920 -17.4..."


In [6]:
tang_gdf.head()

,OBJECTID,Name,Shape_Le_1,Shape_Area,geometry
0,1,Au,507.731122,1.213696e+04,"POLYGON Z ((9746741.997 3480087.392 0.000, 974..."
1,2,Au,22248.346547,6.566996e+06,"POLYGON Z ((10362412.948 5571579.856 0.000, 10..."
2,3,Au,19467.203319,7.109258e+06,"POLYGON Z ((-6976558.709 -1779192.275 0.000, -..."
3,4,Fe,8281.186849,2.323061e+06,"POLYGON Z ((11034240.023 5539946.449 0.000, 11..."
4,5,0,1492.049584,1.231474e+05,"POLYGON Z ((8889342.751 3577269.676 0.000, 888..."


In [7]:
print(maus_gdf.shape)
print(tang_gdf.shape)

(44929, 4)
(74548, 5)


In [8]:
# check how many empty geometries are in the tang_gdf
empty_geometries = tang_gdf[tang_gdf.geometry == None]
len(empty_geometries)

0

In [9]:
# convert projected coordinates in tang to lat long
tang_gdf = tang_gdf.to_crs(epsg=4326)

In [10]:
# Flatten polygon dimensions from 3D to 2D
tang_gdf.geometry = shapely.wkb.loads(shapely.wkb.dumps(tang_gdf.geometry, output_dimension=2))

In [11]:
maus_gdf_filtered = maus_gdf.cx[BBOX[0]:BBOX[2], BBOX[1]:BBOX[3]]
tang_gdf_filtered = tang_gdf.cx[BBOX[0]:BBOX[2], BBOX[1]:BBOX[3]]

In [12]:
maus_gdf_filtered.head()

,ISO3_CODE,COUNTRY_NAME,AREA,geometry
4308,BRA,Brazil,4.526153,"POLYGON ((-50.34230 -6.46640, -50.33970 -6.470..."
4316,BRA,Brazil,4.029746,"POLYGON ((-50.23900 -6.43630, -50.24980 -6.441..."
4329,BRA,Brazil,0.363408,"POLYGON ((-49.87660 -6.20440, -49.87830 -6.208..."
4334,BRA,Brazil,1.585674,"POLYGON ((-50.00340 -6.11420, -50.00800 -6.115..."
4339,BRA,Brazil,0.826450,"POLYGON ((-49.75480 -6.23760, -49.75770 -6.242..."


In [13]:
tang_gdf_filtered.head()

,OBJECTID,Name,Shape_Le_1,Shape_Area,geometry
7687,7688,Placemark,6284.211434,1.395161e+06,"POLYGON ((-50.58005 -6.05403, -50.58030 -6.051..."
7688,7689,Placemark,11521.144792,2.317360e+06,"POLYGON ((-50.57555 -6.02529, -50.57362 -6.025..."
7689,7690,Placemark,17145.217793,8.213193e+06,"POLYGON ((-50.53541 -5.78219, -50.53166 -5.782..."
7690,7691,Placemark,26207.278720,4.201798e+06,"POLYGON ((-50.51591 -5.76025, -50.51560 -5.760..."
7692,7693,Placemark,10317.859600,1.552590e+06,"POLYGON ((-50.30800 -6.10007, -50.30730 -6.100..."


In [23]:
# Create a Leaflet map
m = leafmap.Map(center = [LOCATION[1], LOCATION[0]], zoom=10)
m.add_basemap("satellite")


style = {
    "stroke": True,
    "color": "red",
    "weight": 2,
    "opacity": 1,
    "fill": True,
    "fillColor": "red",
    "fillOpacity": 0.1,
}

# Add the GeoDataFrame to the map
m.add_gdf(maus_gdf_filtered, layer_name="maus_gdf")
m.add_gdf(tang_gdf_filtered, layer_name="tang_gdf", style=style)

# Display the map
m

Basemap can only be one of the following: OpenStreetMap, ROADMAP, SATELLITE, TERRAIN, HYBRID, FWS NWI Wetlands, FWS NWI Wetlands Raster, NLCD 2021 CONUS Land Cover, NLCD 2019 CONUS Land Cover, NLCD 2016 CONUS Land Cover, NLCD 2013 CONUS Land Cover, NLCD 2011 CONUS Land Cover, NLCD 2008 CONUS Land Cover, NLCD 2006 CONUS Land Cover, NLCD 2004 CONUS Land Cover, NLCD 2001 CONUS Land Cover, USGS NAIP Imagery, USGS NAIP Imagery False Color, USGS NAIP Imagery NDVI, USGS Hydrography, USGS 3DEP Elevation, ESA WorldCover 2020, ESA WorldCover 2020 S2 FCC, ESA WorldCover 2020 S2 TCC, ESA WorldCover 2021, ESA WorldCover 2021 S2 FCC, ESA WorldCover 2021 S2 TCC, BaseMapDE.Color, BaseMapDE.Grey, BasemapAT.basemap, BasemapAT.grau, BasemapAT.highdpi, BasemapAT.orthofoto, BasemapAT.overlay, BasemapAT.surface, BasemapAT.terrain, CartoDB.DarkMatter, CartoDB.DarkMatterNoLabels, CartoDB.DarkMatterOnlyLabels, CartoDB.Positron, CartoDB.PositronNoLabels, CartoDB.PositronOnlyLabels, CartoDB.Voyager, CartoDB.Voya

# Overlay a Sentinel Image with the mining polygons

In [3]:
# Download the image 
# Option 1 (Default): Read from Planetary Computer STAC API
api_url="https://planetarycomputer.microsoft.com/api/stac/v1"
bands = ['B04', 'B03', 'B02']
bands_landsat = ['red', 'green', 'blue']

stac_reader = ReadSTAC(api_url=api_url, collection = "sentinel-2-l2a")

# check available items
items = stac_reader.get_items(
    location = LOCATION,
    buffer=10,
    timerange='2020-01-01/2020-12-30',
    max_cloud_cover=10
)

28 Items found.


## Start with only displaying a small Area of Interest

In [4]:
stack = stac_reader.get_stack(items, filter_by="least_cloudy", bands=bands, resolution=10)
stack_stretched = stac_reader.stretch_contrast_stack(stack, upper_percentile=1.0, lower_percentile=0.0)
image = stac_reader.save_stack_as_geotiff(stack_stretched, filename="sentinel_image.tif")

Loading stack...
Found 2 unique tile ids.
Choosing the best 1 items.For unique tile 22MEU, choosing ['S2A_MSIL2A_20200614T134221_R124_T22MEU_20200827T082613'] from [datetime.date(2020, 6, 14)] with [0.008988]% cloud cover
Choosing the best 1 items.For unique tile 22MFU, choosing ['S2B_MSIL2A_20200729T134219_R124_T22MFU_20200818T015550'] from [datetime.date(2020, 7, 29)] with [0.016453]% cloud cover
Stretching contrast...
Saving stack as GeoTIFF under: data/interim/sentinel_image.tif


In [5]:
m = leafmap.Map(center = [LOCATION[1], LOCATION[0]], zoom=10)

m.add_raster(image, layer_name="Image")
m

In [6]:
# add the polygons in the area
m.add_gdf(maus_gdf_filtered, layer_name="maus_gdf")
m.add_gdf(tang_gdf_filtered, layer_name="tang_gdf", style=style)
m

NameError: name 'maus_gdf_filtered' is not defined

## Display the whole S2 Tile

In [ ]:
least_cloudy_item = stac_reader.filter_item(items, filter_by="least_cloudy")

In [ ]:
m = stac_reader.preview_tile_outlines(least_cloudy_item)
m

# Load Raster for mining areas from Maus et al

In [ ]:
# Load the raster for mining areas 
m = leafmap.Map(center = [LOCATION[1], LOCATION[0]], zoom=10)
m.add_raster(MAUS_AREA_RASTER, layer_name="Mining Areas", alpha=0.5)

# Display the map
m

In [ ]:
import rasterio
img = rasterio.open(MAUS_AREA_RASTER)
img

In [ ]:

# Load the GeoTIFF file
with rasterio.open(MAUS_AREA_RASTER) as src:
    for ji, window in src.block_windows(1):
        # Get the geographic coordinates of the pixel
        left, bottom, right, top = src.window_bounds(window)
        print(f"Window bounds: {left, bottom, right, top}")
        # Read the data in the window
        data = src.read(window=window)
        print(f"Data shape: {data.shape}")

        break


In [ ]:
src = rasterio.open(MAUS_AREA_RASTER)
array = src.read(1)
print(array.shape)
array

In [ ]:
# set all values less than 0 to 0 
array[array < 0] = 0
array

In [ ]:
from matplotlib import pyplot
pyplot.imshow(array, cmap='pink')
pyplot.show()

In [ ]:
src.transform

In [ ]:
import numpy as np
from tqdm import tqdm
src = rasterio.open(MAUS_AREA_RASTER)
array = src.read(1)

# Get the transformation matrix
transform = src.transform

# Create an empty list to store the bounding boxes
bounding_boxes = []
mining_area = []

# Iterate over the pixels in the raster
# only record bounding box if they have over 0.5 square km of mining area (out of a total area per square of 78.41 sq.km)
for x in tqdm(range(src.width)):
    for y in range(src.height):
        if array[y, x] > 0.5:
            # Get the pixel's bounding box
            # The bounding box is defined by the pixel's top-left and bottom-right corners
            top_left = transform * (x, y)
            bottom_right = transform * (x + 1, y + 1)
            bounding_box = [top_left[0], bottom_right[1], bottom_right[0], top_left[1]]
            
            # Add the bounding box to the list
            bounding_boxes.append(bounding_box)

            # add the mining area to the list
            mining_area.append(array[y, x])


In [ ]:
len(bounding_boxes)

In [ ]:
# Create a GeoDataFrame from the bounding boxes and the area
gdf = gpd.GeoDataFrame(geometry=[box(*bbox) for bbox in bounding_boxes], crs="EPSG:4326")
gdf["mining_area"] = mining_area
gdf

In [ ]:

# Create a Map
m = leafmap.Map(center=[(bounding_boxes[0][1] + bounding_boxes[0][3]) / 2, 
                        (bounding_boxes[0][0] + bounding_boxes[0][2]) / 2], zoom=2)

# Add the GeoDataFrame to the map
m.add_gdf(gdf.iloc[0:100,:], layer_name="bounding_boxes")

# Display the map
m

In [ ]:
# save the bounding boxes as a geopackage file
gdf.to_file("/workspaces/mine-segmentation/data/interim/mining_areas.gpkg", driver="GPKG")

# Sample random mining tile, plot 2019 sentinel image, and add both mining datasets to it. 

In [ ]:
# read gdf
gdf = gpd.read_file("/workspaces/mine-segmentation/data/interim/mining_areas.gpkg")

In [ ]:
import random

# Set the random seed for reproducibility
random.seed(123)

# Sample a random mining tile
random_tile = gdf.sample(n=1, random_state=random.randint(0, 100))

# Get the geometry of the random tile
tile_geometry = random_tile['geometry'].values[0]

bbox = tile_geometry.bounds

# get the least cloudy sentinel image for the tile
items = stac_reader.get_items(
    bbox=bbox,
    timerange='2019-01-01/2019-12-31',
    max_cloud_cover=10
)

stack = stac_reader.get_stack(items, filter_by="least_cloudy", bands=bands, resolution=10)
stack_stretched = stac_reader.stretch_contrast_stack(stack, upper_percentile=1.0, lower_percentile=0.0)
image = stac_reader.save_stack_as_geotiff(stack_stretched, filename="sentinel_image.tif")

# Create a Map
m = leafmap.Map(center=[tile_geometry.centroid.y, tile_geometry.centroid.x], zoom=2)

# add the image
m.add_image(image)

# Filter the polygons that are included 
maus_gdf_filtered = maus_gdf.cx[bbox[0]:bbox[2], bbox[1]:bbox[3]]
tang_gdf_filtered = tang_gdf.cx[bbox[0]:bbox[2], bbox[1]:bbox[3]]

style = {
    "stroke": True,
    "color": "red",
    "weight": 2,
    "opacity": 1,
    "fill": True,
    "fillColor": "red",
    "fillOpacity": 0.1,
}

# Display the filtered gdfs
m.add_gdf(maus_gdf_filtered, layer_name="maus_gdf")
m.add_gdf(tang_gdf_filtered, layer_name="tang_gdf", style=style)

m